In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print('GPU acceleration is available and will be used :-)')
else:
    DEVICE = torch.device("cpu")
    print('GPU is not available. If you are using Google Colab, change the runtime to GPU, otherwise training will '
          'take too long.')

GPU is not available. If you are using Google Colab, change the runtime to GPU, otherwise training will take too long.


In [75]:
!sh download_data.sh

--2021-01-27 21:16:43--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.251.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.251.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘./data/personachat_self_original.json.1’

personachat_self_or 100%[===================>] 200.13M  5.70MB/s    in 36s     

2021-01-27 21:17:19 (5.60 MB/s) - ‘./data/personachat_self_original.json.1’ saved [209850483/209850483]



In [65]:
from data_util import generate_train_validation_test_files

generate_train_validation_test_files()

0 lines removed
Train set 131,438
0 lines removed
Test set 7,801


In [3]:
from data_util import DataWrapper

# Dataset objects
dw = DataWrapper()
train_ds, val_ds, test_ds = dw.get_datasets(
    train_size=9,
    val_size=99,
    use_glove=True
)
print(f'Train set size: {len(train_ds):,}')
print(f'Validation set size: {len(val_ds):,}')
print('Vocab size: ', dw.vocab_size)

/Users/paulabartabajo/miniconda3/envs/chatbot_py3.6/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/paulabartabajo/miniconda3/envs/chatbot_py3.6/lib/python3.6/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/Users/paulabartabajo/miniconda3/envs/chatbot_py3.6/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class w

Train set size: 9
Validation set size: 99
Vocab size:  94


In [4]:
# DataLoader objects
train_iter, val_iter, test_iter = dw.get_dataloaders(
    train_ds, val_ds, test_ds,
    batch_size=2400,
    device=DEVICE
)

x = next(iter(train_iter))
print('Example \n-------')
print(x.src[0])
print(x.src[1])

Example 
-------
tensor([[ 2, 33, 28,  ...,  0, 10,  3],
        [ 2, 33, 28,  ...,  1,  1,  1],
        [ 2, 33, 28,  ...,  1,  1,  1],
        ...,
        [ 2, 33, 28,  ...,  1,  1,  1],
        [ 2, 33, 28,  ...,  1,  1,  1],
        [ 2, 33, 28,  ...,  1,  1,  1]])
tensor([194, 166, 140, 116,  82,  52,  33,  23,  10])


/Users/paulabartabajo/miniconda3/envs/chatbot_py3.6/lib/python3.6/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/paulabartabajo/miniconda3/envs/chatbot_py3.6/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [37]:
%autoreload 2
from model import Seq2seqRNN, count_parameters

vocab_size = dw.vocab_size
embedding_dim = dw.embedding_dim
hidden_dim = 256
n_layers = 3
n_directions_encoder = 2
model = Seq2seqRNN(vocab_size,
                   embedding_dim,
                   hidden_dim,
                   n_layers,
                   n_directions_encoder,
                   dropout=0.2,
                   pretrained_embeddings=dw.embeddings,
                   freeze_embeddings=False)

print(f'The model has {count_parameters(model):,} parameters')

The model has 5,339,926 parameters


In [50]:
%autoreload 2
from train import Seq2seqRNNTrainer

trainer = Seq2seqRNNTrainer(model,
                            train_iter,
                            val_iter,
                            learning_rate=3e-4,
                            pad_token_id=dw.pad_token_id,
                            gradient_clip=99999,
                            teacher_forcing=0.5,
                            checkpoint_dir='./checkpoints')

n_epochs = 3
trainer.train(n_epochs)

# trainer.load_checkpoint()

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

Epoch: 000, Train loss: 3.9353, Val loss: 3.5527, Train ppl: 51.2, Val ppl: 34.9
checkpoints/b20de6de-64a6-11eb-9720-acbc32b70c09/1.ckpt was saved
checkpoints/b20de6de-64a6-11eb-9720-acbc32b70c09/params.json file was saved
> /Users/paulabartabajo/src/online-courses/advanced-nlp-chatbot/train.py(186)download_artifacts()
    184 
    185     # download tar file
--> 186     from google.colab import files
    187     files.download(tar)
    188     print(f'Downloaded {tar}')

ipdb> tar
ipdb> q


BdbQuit: 